# Imbalanced data

In [ ]:
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = [12, 8]

sklearn.set_config(display='diagram')

## Load Mammography Data

In [ ]:
from sklearn.datasets import fetch_openml

In [ ]:
mammography = fetch_openml(data_id=310)
X, y = mammography.data, mammography.target

In [ ]:
y = (y == '1').astype(int)

In [ ]:
np.bincount(y)

## Split data into train test split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, random_state=0)

### Base models
#### Linear model

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression

In [ ]:
base_log_reg = LogisticRegression(random_state=42)
cv_results = cross_validate(base_log_reg,
                            X_train, y_train, scoring=['roc_auc', 'average_precision'])

In [ ]:
log_reg_scores

In [ ]:
log_reg_base_auc = log_reg_scores['test_roc_auc'].mean()
log_reg_base_auc

In [ ]:
log_reg_base_ap = log_reg_scores['test_average_precision'].mean()
log_reg_base_ap

In [ ]:
def compute_metrics(estimator):
    cv_results = cross_validate(estimator,
                                X_train, y_train, scoring=['roc_auc', 'average_precision'])
    return {
        "auc": cv_results["test_roc_auc"].mean(),
        "average_precision": cv_results["test_average_precision"].mean(),
    }

In [ ]:
base_log_reg_metrics = compute_metrics(base_log_reg)
base_log_reg_metrics

#### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
base_rf = RandomForestClassifier(random_state=42, n_jobs=-1)

In [ ]:
base_rf_metrics = compute_metrics(base_rf)
base_rf_metrics

### Imbalance-learn sampler

#### Under sampler

In [ ]:
np.bincount(y_train)

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
under_sampler = RandomUnderSampler(random_state=42)

In [ ]:
X_train_subsample, y_train_subsample = under_sampler.fit_resample(X_train, y_train)

In [ ]:
X_train.shape

In [ ]:
X_train_subsample.shape

In [ ]:
np.bincount(y_train_subsample)

#### Oversampling

In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [ ]:
over_sampler = RandomOverSampler(random_state=42)

In [ ]:
X_train_subsample, y_train_subsample = over_sampler.fit_resample(X_train, y_train)

In [ ]:
X_train_subsample.shape

In [ ]:
np.bincount(y_train_subsample)

## Pipelines with imblean

### Linear model with under sampling

In [ ]:
from imblearn.pipeline import make_pipeline as make_imb_pipeline

In [ ]:
under_log_reg = make_imb_pipeline(
    RandomUnderSampler(random_state=42), LogisticRegression(random_state=42))

In [ ]:
base_log_reg_metrics

In [ ]:
compute_metrics(under_log_reg)

### Random Forest with under sampling

In [ ]:
under_rf = make_imb_pipeline(
    RandomUnderSampler(random_state=42), RandomForestClassifier(random_state=42))

In [ ]:
base_rf_metrics

In [ ]:
compute_metrics(under_rf)

### Linear model with over sampling

In [ ]:
over_log_reg = make_imb_pipeline(
    RandomOverSampler(), LogisticRegression(random_state=42))

In [ ]:
base_log_reg_metrics

In [ ]:
compute_metrics(over_log_reg)

## Exercise 1

1. Use `make_imb_pipeline` with `RandomOverSampler` to create a pipline with random forset called `over_rf`.
1. Compute our metrics using `compute_metrics`.

In [ ]:
# %load solutions/02-ex01-solutions.py

## Plotting curves for logistic regression

In [ ]:
base_log_reg.fit(X_train, y_train)
under_log_reg.fit(X_train, y_train)
over_log_reg.fit(X_train, y_train);

In [ ]:
base_log_reg.score(X_test, y_test)

### Plotting

In [ ]:
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import plot_roc_curve

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 8))
plot_roc_curve(base_log_reg, X_test, y_test, ax=ax1, name="original")
plot_roc_curve(under_log_reg, X_test, y_test, ax=ax1, name="undersampling")
plot_roc_curve(over_log_reg, X_test, y_test, ax=ax1, name="oversampling")

plot_precision_recall_curve(base_log_reg, X_test, y_test, ax=ax2, name="original")
plot_precision_recall_curve(under_log_reg, X_test, y_test, ax=ax2, name="undersampling")
plot_precision_recall_curve(over_log_reg, X_test, y_test, ax=ax2, name="oversampling");

## Exercise 2

1. Train the three random forest models, `base_rf`, `under_rf`, `over_rf`.
1. Plot the roc and precision recall for the three random forest models.

In [ ]:
# %load solutions/02-ex02-solutions.py

## Class-Weights

#### Linear model with class weights

In [ ]:
class_weight_log_reg = LogisticRegression(class_weight='balanced', random_state=42)
class_weight_log_reg.fit(X_train, y_train)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 8))
plot_roc_curve(base_log_reg, X_test, y_test, ax=ax1, name="original")
plot_roc_curve(class_weight_log_reg, X_test, y_test, ax=ax1, name="class-weighted")

plot_precision_recall_curve(base_log_reg, X_test, y_test, ax=ax2, name="original")
plot_precision_recall_curve(class_weight_log_reg, X_test, y_test, ax=ax2, name="class-weighted")

#### Random forest with class weights 

In [ ]:
class_weight_rf = RandomForestClassifier(class_weight='balanced', random_state=42)
class_weight_rf.fit(X_train, y_train)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 8))
plot_roc_curve(base_rf, X_test, y_test, ax=ax1, name="original")
plot_roc_curve(class_weight_rf, X_test, y_test, ax=ax1, name="class-weighted")

plot_precision_recall_curve(base_rf, X_test, y_test, ax=ax2, name="original")
plot_precision_recall_curve(class_weight_rf, X_test, y_test, ax=ax2, name="class-weighted")

## Ensemble Resampling

In [ ]:
from imblearn.ensemble import BalancedRandomForestClassifier

In [ ]:
balanced_rf = BalancedRandomForestClassifier(random_state=0)
balanced_rf.fit(X_train, y_train)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 8))
plot_roc_curve(base_rf, X_test, y_test, ax=ax1, name="original")
plot_roc_curve(under_rf, X_test, y_test, ax=ax1, name="undersampling")
plot_roc_curve(over_rf, X_test, y_test, ax=ax1, name="oversampling")
plot_roc_curve(balanced_rf, X_test, y_test, ax=ax1, name="balanced bagging")

plot_precision_recall_curve(base_rf, X_test, y_test, ax=ax2, name="original")
plot_precision_recall_curve(under_rf, X_test, y_test, ax=ax2, name="undersampling")
plot_precision_recall_curve(over_rf, X_test, y_test, ax=ax2, name="oversampling");
plot_precision_recall_curve(balanced_rf, X_test, y_test, ax=ax2, name="balanced bagging")

## SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

X_train_smote.shape

In [ ]:
np.bincount(y_train_smote)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 8))
sorting = np.argsort(y_train)

axes[0].set_title("Original")
axes[0].scatter(X_train.iloc[sorting, 3], X_train.iloc[sorting, 4], c=plt.cm.tab10(y_train.iloc[sorting]), alpha=.3, s=2)

axes[1].set_title("SMOTE")
axes[1].scatter(X_train_smote.iloc[:, 3], X_train_smote.iloc[:, 4], c=plt.cm.tab10(y_train_smote), alpha=.1, s=2)

In [ ]:
log_reg_base_metrics

In [ ]:
smote_log_reg = make_imb_pipeline(
    SMOTE(random_state=42), LogisticRegression(random_state=42))
compute_metrics(smote_log_reg)

In [ ]:
base_rf_metrics

In [ ]:
smote_rf = make_imb_pipeline(SMOTE(random_state=42), RandomForestClassifier(random_state=42, n_jobs=-1))
compute_metrics(smote_rf)

## Plotting all the version of random forest

In [ ]:
smote_rf.fit(X_train, y_train)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 8))
plot_roc_curve(base_rf, X_test, y_test, ax=ax1, name="original")
plot_roc_curve(under_rf, X_test, y_test, ax=ax1, name="undersampling")
plot_roc_curve(over_rf, X_test, y_test, ax=ax1, name="oversampling")
plot_roc_curve(balanced_rf, X_test, y_test, ax=ax1, name="balanced bagging")
plot_roc_curve(smote_rf, X_test, y_test, ax=ax1, name="smote")

plot_precision_recall_curve(base_rf, X_test, y_test, ax=ax2, name="original")
plot_precision_recall_curve(under_rf, X_test, y_test, ax=ax2, name="undersampling")
plot_precision_recall_curve(over_rf, X_test, y_test, ax=ax2, name="oversampling");
plot_precision_recall_curve(balanced_rf, X_test, y_test, ax=ax2, name="balanced bagging")
plot_precision_recall_curve(smote_rf, X_test, y_test, ax=ax2, name="smote")

## Exercise 3

1. Train a `HistGradientBoostingClassifer` on the training set.

```py
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifer
```

1. Construct a pipline with `SMOTE` and `HistGradientBoostingClassifer` fit it on the training set.
1. Plot the ROC and PR curves between the two models.

In [ ]:
# %load solutions/02-ex03-solutions.py